In [142]:
import numpy as np

In [184]:
train = open('../data/processed/train.word','r').readlines()
freqsf = open('../data/processed/freqs.txt', 'r').readlines()

In [185]:
s1s = [l.split('\t')[1].split(' ') for l in train]
s2s = [l.split('\t')[2].split(' ') for l in train]
labels = [int(l.split('\t')[3]) for l in train]
words = [s.split(' ')[0] for s in freqsf if s[0] != ' ']
freqs = [int(s.strip().split(' ')[1]) for s in freqsf  if s[0] != ' ']

In [186]:
count = [0,0]
for l in labels:
    count[l] += 1
print(count, float(count[0])/count[1])
ratio = float(count[0])/count[1]

[27676, 7735] 3.578021978021978


In [222]:
def analyze_boolean_feature(func, verbose=False):
    ratio = float(count[0])/count[1]
    eps = 1e-6
#     ratio = 1.
    n = 0
    results = [[0, 0], [0, 0]]
    hit = [0,0]
    cls = [0,0]
    for s1, s2, l in zip(s1s, s2s, labels):
        n += 1
        r = func(s1, s2, l)
        results[r][l] += 1
        hit[r] += 1
        cls[l] += 1
    PCF = np.zeros((2,2)) # P(C|F)
    PFC = np.zeros((2,2)) # P(F|C)
    for c in [0,1]:
        for f in [0,1]: 
            PCF[c,f] = results[f][c]/float(hit[f] + eps) 
    for c in [0,1]:
        for f in [0,1]: 
            PFC[f,c] = results[f][c]/float(cls[c] + eps) 
    if verbose:
        print("# C            0         1")
        print("# P(C|F=0)   {0:.3f}      {1:.3f}".format(PCF[0,0], PCF[1,0]))
        print("# P(C|F=1)   {0:.3f}      {1:.3f}".format(PCF[0,1], PCF[1,1]))
#         print("# Feature=0  {0:.3f}      {1:.3f}".format(results[0][0]/float(n), results[0][1]/float(n)*ratio))
#         print("# Feature=1  {0:.3f}      {1:.3f}".format(results[1][0]/float(n), results[1][1]/float(n)*ratio))  
        print("# P(F=0|C)   {0:.3f}      {1:.3f}".format(PFC[0,0], PFC[0,1]))
        print("# P(F=1|C)   {0:.3f}      {1:.3f}".format(PFC[1,0], PFC[1,1]))
    return PCF, PFC

In [240]:
def func(s1, s2, l):
    # 只在一个句子中有该词
    word = u'吗' # 要注意同义词, 手动设置近义疑问词
    if (word in s1 and word not in s2) or (word in s2 and word not in s1):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)
# 花呗
#              0         1
# Feature=0  0.787      0.815
# Feature=1  0.931      0.265
# Feature=0  0.756      0.782
# Feature=1  0.038      0.011

# 借呗
#              0         1
# Feature=0  0.786      0.822
# Feature=1  0.946      0.208
# Feature=0  0.749      0.783
# Feature=1  0.044      0.010

# C            0         1
# P(C|F=0)   0.768      0.232
# P(C|F=1)   0.838      0.162
# P(F=0|C)   0.797      0.859
# P(F=1|C)   0.203      0.141


(array([[ 0.76841335,  0.83782978],
        [ 0.23158665,  0.16217022]]), array([[ 0.79689984,  0.85934066],
        [ 0.20310016,  0.14065934]]))

In [241]:
def build_func(word):
    def func(s1, s2, l):
        # 只在一个句子中有该词
        if (word in s1 and word not in s2) or (word in s2 and word not in s1):
            return 1
        else:
            return 0
    return func
cands = []
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.05 and PFC[1,0] > 0.005:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}, P(F=1|C=0)={3:.3f}, P(F=1|C=1)={4:.3f}"\
                                  .format(w, PCF[0,0], PCF[0,1], PFC[1,0], PFC[1,1]))
            cands.append(w)
        
    

花呗	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.927, P(F=1|C=0)=0.045, P(F=1|C=1)=0.013
借呗	 P(C=0|F=0)=0.774, P(C=0|F=1)=0.956, P(F=1|C=0)=0.051, P(F=1|C=1)=0.008
吗	 P(C=0|F=0)=0.768, P(C=0|F=1)=0.838, P(F=1|C=0)=0.203, P(F=1|C=1)=0.141
还	 P(C=0|F=0)=0.770, P(C=0|F=1)=0.837, P(F=1|C=0)=0.189, P(F=1|C=1)=0.132
还款	 P(C=0|F=0)=0.775, P(C=0|F=1)=0.826, P(F=1|C=0)=0.134, P(F=1|C=1)=0.101
蚂蚁	 P(C=0|F=0)=0.792, P(C=0|F=1)=0.723, P(F=1|C=0)=0.141, P(F=1|C=1)=0.194
额度	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.859, P(F=1|C=0)=0.068, P(F=1|C=1)=0.040
分期	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.891, P(F=1|C=0)=0.053, P(F=1|C=1)=0.023
开通	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.829, P(F=1|C=0)=0.060, P(F=1|C=1)=0.044
是	 P(C=0|F=0)=0.775, P(C=0|F=1)=0.831, P(F=1|C=0)=0.122, P(F=1|C=1)=0.088
没有	 P(C=0|F=0)=0.778, P(C=0|F=1)=0.838, P(F=1|C=0)=0.070, P(F=1|C=1)=0.048
有	 P(C=0|F=0)=0.777, P(C=0|F=1)=0.836, P(F=1|C=0)=0.087, P(F=1|C=1)=0.061
付款	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.833, P(F=1|C=0)=0.051, P(F=1|C=1)=0.037
多少	 P(C=0|F=0)=0.779, P(C=0|F

In [237]:
cands

['花呗',
 '借呗',
 '分期',
 '多少',
 '逾期',
 '提前',
 '银行卡',
 '余额',
 '淘宝',
 '临时',
 '影响',
 '利息',
 '信用卡',
 '手续费',
 '扣',
 '是不是',
 '最低',
 '宝',
 '不是',
 '算',
 '为啥',
 '积分',
 '电费',
 '限制',
 '二维码',
 '更改',
 '方式',
 '别人',
 '先']

In [163]:
def contains(words, l):
    for w in words:
        if w in l:
            return True
    return False

def func(s1, s2, l):
    # 只在一个句子中有该类词
    word = [u'借呗'] # 要注意同义词, 手动设置近义疑问词
    if (contains(word, s1) and not contains(word, s2)) or (contains(word, s2) and not contains(word, s1)):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func)

In [214]:
def func(s1, s2, l):
    # 两个句子里都有该词
    word = u'退款'
    if word in s1 and word in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)

# C            0         1
# P(C|F=0)   0.783      0.217
# P(C|F=1)   0.667      0.333
# P(F=0|C)   0.986      0.975
# P(F=1|C)   0.014      0.025


(array([[ 0.78345485,  0.66666667],
        [ 0.21654515,  0.33333333]]), array([[ 0.98619743,  0.97530705],
        [ 0.01380257,  0.02469295]]))

In [233]:
def build_func(word):
    def func(s1, s2, l):
        # 两个句子里都有该词
        if word in s1 and word in s2:
            return 1
        else:
            return 0
    return func
cands = []
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.05 and PFC[1,0] > 0.005:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}, P(F=1|C=0)={3:.3f}, P(F=1|C=1)={4:.3f}"\
                                  .format(w, PCF[0,0], PCF[0,1], PFC[1,0], PFC[1,1]))
            cands.append(w)
        
    

*	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.686, P(F=1|C=0)=0.017, P(F=1|C=1)=0.027
没有	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.720, P(F=1|C=0)=0.021, P(F=1|C=1)=0.029
不	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.711, P(F=1|C=0)=0.013, P(F=1|C=1)=0.019
在	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.662, P(F=1|C=0)=0.010, P(F=1|C=1)=0.018
不了	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.638, P(F=1|C=0)=0.010, P(F=1|C=1)=0.020
支付	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.868, P(F=1|C=0)=0.011, P(F=1|C=1)=0.006
月	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.689, P(F=1|C=0)=0.009, P(F=1|C=1)=0.015
到	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.639, P(F=1|C=0)=0.009, P(F=1|C=1)=0.019
显示	 P(C=0|F=0)=0.782, P(C=0|F=1)=0.723, P(F=1|C=0)=0.012, P(F=1|C=1)=0.016
会	 P(C=0|F=0)=0.782, P(C=0|F=1)=0.710, P(F=1|C=0)=0.009, P(F=1|C=1)=0.013
退款	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.667, P(F=1|C=0)=0.014, P(F=1|C=1)=0.025
时候	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.696, P(F=1|C=0)=0.011, P(F=1|C=1)=0.017
关闭	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.644, P(F=1|C=0)=0.010, P(F=1|C=1)=0.020
如何	 P(C=0|F=0)=0.782, P(C=0|F=1

In [206]:
def func(s1, s2, l):
    # 两个句子里都没有该词
    word = u'花呗'
    if word not in s1 and word not in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)

# C            0         1
# P(C|F=0)   0.790      0.210
# P(C|F=1)   0.759      0.241
# P(F=0|C)   0.733      0.697
# P(F=1|C)   0.267      0.303


(array([[ 0.79011336,  0.75903491],
        [ 0.20988664,  0.24096509]]), array([[ 0.73287325,  0.69657401],
        [ 0.26712675,  0.30342599]]))

In [239]:
def build_func(word):
    def func(s1, s2, l):
        # 两个句子里都有该词
        if not(word not in s1 and word not in s2):
            return 1
        else:
            return 0
    return func
cands = []
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.05 and PFC[1,0] > 0.005:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}, P(F=1|C=0)={3:.3f}, P(F=1|C=1)={4:.3f}"\
                                  .format(w, PCF[0,0], PCF[0,1], PFC[1,0], PFC[1,1]))
            cands.append(w)
        
    

吗	 P(C=0|F=0)=0.764, P(C=0|F=1)=0.823, P(F=1|C=0)=0.314, P(F=1|C=1)=0.242
额度	 P(C=0|F=0)=0.773, P(C=0|F=1)=0.836, P(F=1|C=0)=0.142, P(F=1|C=1)=0.099
分期	 P(C=0|F=0)=0.775, P(C=0|F=1)=0.841, P(F=1|C=0)=0.110, P(F=1|C=1)=0.074
是	 P(C=0|F=0)=0.774, P(C=0|F=1)=0.827, P(F=1|C=0)=0.144, P(F=1|C=1)=0.108
逾期	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.839, P(F=1|C=0)=0.047, P(F=1|C=1)=0.032
提前	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.844, P(F=1|C=0)=0.044, P(F=1|C=1)=0.029
银行卡	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.836, P(F=1|C=0)=0.035, P(F=1|C=1)=0.024
收款	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.832, P(F=1|C=0)=0.035, P(F=1|C=1)=0.025
余额	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.867, P(F=1|C=0)=0.036, P(F=1|C=1)=0.020
自动	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.836, P(F=1|C=0)=0.034, P(F=1|C=1)=0.024
如何	 P(C=0|F=0)=0.785, P(C=0|F=1)=0.705, P(F=1|C=0)=0.036, P(F=1|C=1)=0.053
和	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.841, P(F=1|C=0)=0.041, P(F=1|C=1)=0.028
都	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.838, P(F=1|C=0)=0.038, P(F=1|C=1)=0.026
临时	 P(C=0|F=0)=0.779, P(C=0|